In [ ]:
# import
import os
import numpy as np
import csv

In [ ]:
# import my google drive
from google.colab import drive
drive.mount('/content/gdrive')

# read fasta files  
fasta_file_path = 'gdrive/My Drive/data sample/NetMHCpan/'
processed_file_path = 'gdrive/My Drive/data sample/NetMHCpan_processed/'
print('list of files', os.listdir(fasta_file_path))

# loading training set with ba data
file_name_keyword = 'ba'
all_allele = list()
allele_list = list()
sequence_list = list()
ba_list = list()

for single_file in [x for x in os.listdir(fasta_file_path) if file_name_keyword in x and x.startswith('f')]:
  print('Processing file...', single_file)
  with open(fasta_file_path + single_file) as f:
    for line in f.readlines():
      if len(line.split()) == 4:
        [sequence, ba, allele, _] = line.split()
        allele_list.append(allele)
        if not allele in all_allele:
          all_allele.append(allele)
        sequence_list.append(sequence)
        ba_list.append(float(ba))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
list of files ['f001_ba.txt', 'f000_ba.txt', 'f002_ba.txt', 'f003_ba.txt', 'c000_ba.txt', 'f004_ba.txt', 'c001_ba.txt', 'c002_ba.txt', 'c003_ba.txt', 'c004_ba.txt', 'c002_el.txt', 'c001_el.txt', 'c003_el.txt', 'c000_el.txt', 'c004_el.txt', 'f000_el.txt', 'f003_el.txt', 'f001_el.txt', 'f002_el.txt', 'f004_el.txt']
Processing file... f001_ba.txt
Processing file... f000_ba.txt
Processing file... f002_ba.txt
Processing file... f003_ba.txt
Processing file... f004_ba.txt


In [ ]:
allele_cnt = dict()
for curr_allele in all_allele:
  allele_cnt[curr_allele] = allele_list.count(curr_allele)
allele_cnt = sorted(allele_cnt.items(), key=lambda x:x[1], reverse=True)
print(allele_cnt)

[('HLA-A02:01', 48568), ('HLA-A03:01', 29376), ('HLA-A11:01', 25552), ('HLA-A02:03', 25184), ('HLA-A31:01', 22812), ('HLA-A02:06', 22488), ('HLA-A68:02', 22160), ('HLA-A01:01', 19120), ('HLA-B07:02', 18540), ('HLA-A26:01', 18108), ('HLA-B15:01', 17764), ('HLA-A02:02', 16608), ('H-2-Kb', 15984), ('H-2-Db', 15220), ('HLA-A68:01', 15160), ('HLA-B27:05', 14684), ('HLA-A33:01', 14368), ('HLA-B08:01', 14136), ('HLA-B40:01', 13752), ('HLA-B58:01', 13356), ('HLA-A24:02', 13096), ('HLA-B35:01', 12824), ('HLA-A30:01', 11684), ('HLA-B51:01', 11588), ('HLA-B57:01', 11588), ('HLA-B18:01', 11308), ('HLA-A29:02', 11128), ('HLA-A23:01', 10548), ('Mamu-A01', 10356), ('HLA-A69:01', 10240), ('HLA-B44:02', 9252), ('HLA-A30:02', 8204), ('HLA-B46:01', 7216), ('HLA-B39:01', 7140), ('HLA-B53:01', 6872), ('Mamu-B17', 6592), ('HLA-B15:17', 5776), ('HLA-B44:03', 5616), ('HLA-A24:03', 5516), ('Mamu-A02', 5416), ('Mamu-A11', 5040), ('HLA-A02:19', 4972), ('HLA-B54:01', 4808), ('HLA-A02:12', 4724), ('HLA-A80:01', 46

In [ ]:
allele_used = []

for (curr_allele, cnt) in allele_cnt:
  if cnt < 20000:
    break
  allele_used.append(curr_allele)
  curr_sequence_list = []
  curr_label_list = []
  print("Processing "+curr_allele)
  index = 0
  cnt = 0
  for allele in allele_list:
    if allele == curr_allele:
      cnt += 1
      if len(sequence_list[index]) < 20:
        curr_sequence_list.append(sequence_list[index])
        if ba_list[index] > 0.426:
          curr_label_list.append(1)
        else:
          curr_label_list.append(0)
    index += 1
  print("Data points: "+str(len(curr_sequence_list)))
  print("Original Data points: "+str(cnt))
  print("ratio: "+str((cnt-len(curr_sequence_list))*100/cnt))

  f = open(processed_file_path+'train_'+curr_allele+'.tsv', 'w', encoding='utf-8', newline='')
  wr = csv.writer(f, delimiter='\t')
  title = np.arange(2).tolist()
  title[0] = "peptide"
  title[1] = "label_num"
  wr.writerow(title)

  for index in range(len(curr_label_list)):
    data = []
    data.append(curr_sequence_list[index])
    data.append(curr_label_list[index])
    wr.writerow(data)

  f.close()

Processing HLA-A02:01
Data points: 48564
Original Data points: 48568
ratio: 0.00823587547356284
Processing HLA-A03:01
Data points: 29376
Original Data points: 29376
ratio: 0.0
Processing HLA-A11:01
Data points: 25552
Original Data points: 25552
ratio: 0.0
Processing HLA-A02:03
Data points: 25184
Original Data points: 25184
ratio: 0.0
Processing HLA-A31:01
Data points: 22812
Original Data points: 22812
ratio: 0.0
Processing HLA-A02:06
Data points: 22488
Original Data points: 22488
ratio: 0.0
Processing HLA-A68:02
Data points: 22160
Original Data points: 22160
ratio: 0.0


In [ ]:
allele_used = []

curr_sequence_list = []
curr_label_list = []
curr_allele_list = []

for (curr_allele, cnt) in allele_cnt:
  if cnt < 20000:
    break
  allele_used.append(curr_allele)
  print("Processing "+curr_allele)
  index = 0
  for allele in allele_list:
    if allele == curr_allele:
      if len(sequence_list[index]) < 20:
        curr_sequence_list.append(sequence_list[index])
        if ba_list[index] > 0.426:
          curr_label_list.append(1)
        else:
          curr_label_list.append(0)
        curr_allele_list.append(curr_allele)
    index += 1
  print("Data points: "+str(len(curr_sequence_list)))

f = open(processed_file_path+'train_all.tsv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f, delimiter='\t')
title = np.arange(3).tolist()
title[0] = "peptide"
title[1] = "allele"
title[2] = "label_num"
wr.writerow(title)

for index in range(len(curr_label_list)):
  data = []
  data.append(curr_sequence_list[index])
  data.append(curr_allele_list[index])
  data.append(curr_label_list[index])
  wr.writerow(data)

f.close()

Processing HLA-A02:01
Data points: 48564
Processing HLA-A03:01
Data points: 77940
Processing HLA-A11:01
Data points: 103492
Processing HLA-A02:03
Data points: 128676
Processing HLA-A31:01
Data points: 151488
Processing HLA-A02:06
Data points: 173976
Processing HLA-A68:02
Data points: 196136


In [ ]:
# loading training set with ba data
file_name_keyword = 'ba'
all_allele = list()
allele_list = list()
sequence_list = list()
ba_list = list()

for single_file in [x for x in os.listdir(fasta_file_path) if file_name_keyword in x and x.startswith('c')]:
  print('Processing file...', single_file)
  with open(fasta_file_path + single_file) as f:
    for line in f.readlines():
      if len(line.split()) == 4:
        [sequence, ba, allele, _] = line.split()
        if allele in allele_used:
          allele_list.append(allele)
          if not allele in all_allele:
            all_allele.append(allele)
          sequence_list.append(sequence)
          ba_list.append(float(ba))

Processing file... c000_ba.txt
Processing file... c001_ba.txt
Processing file... c002_ba.txt
Processing file... c003_ba.txt
Processing file... c004_ba.txt


In [ ]:
allele_cnt = dict()
for curr_allele in all_allele:
  allele_cnt[curr_allele] = allele_list.count(curr_allele)
allele_cnt = sorted(allele_cnt.items(), key=lambda x:x[1], reverse=True)
print(allele_cnt)

[('HLA-A02:01', 12142), ('HLA-A03:01', 7344), ('HLA-A11:01', 6388), ('HLA-A02:03', 6296), ('HLA-A31:01', 5703), ('HLA-A02:06', 5622), ('HLA-A68:02', 5540)]


In [ ]:
for (curr_allele, cnt) in allele_cnt:
  curr_sequence_list = []
  curr_label_list = []
  print("Processing "+curr_allele)
  index = 0
  for allele in allele_list:
    if allele == curr_allele and len(sequence_list[index]) < 14:
      curr_sequence_list.append(sequence_list[index])
      if ba_list[index] > 0.426:
        curr_label_list.append(1)
      else:
        curr_label_list.append(0)
    index += 1
  print("Data points: "+str(len(curr_sequence_list)))

  f = open(processed_file_path+'test_'+curr_allele+'.tsv', 'w', encoding='utf-8', newline='')
  wr = csv.writer(f, delimiter='\t')
  title = np.arange(2).tolist()
  title[0] = "peptide"
  title[1] = "label_num"
  wr.writerow(title)

  for index in range(len(curr_label_list)):
    data = []
    data.append(curr_sequence_list[index])
    data.append(curr_label_list[index])
    wr.writerow(data)

  f.close()

Processing HLA-A02:01
Data points: 12117
Processing HLA-A03:01
Data points: 7318
Processing HLA-A11:01
Data points: 6372
Processing HLA-A02:03
Data points: 6277
Processing HLA-A31:01
Data points: 5689
Processing HLA-A02:06
Data points: 5600
Processing HLA-A68:02
Data points: 5520


In [ ]:
allele_used = []

curr_sequence_list = []
curr_label_list = []
curr_allele_list = []

for (curr_allele, cnt) in allele_cnt:
  allele_used.append(curr_allele)
  print("Processing "+curr_allele)
  index = 0
  for allele in allele_list:
    if allele == curr_allele:
      if len(sequence_list[index]) < 20:
        curr_sequence_list.append(sequence_list[index])
        if ba_list[index] > 0.426:
          curr_label_list.append(1)
        else:
          curr_label_list.append(0)
        curr_allele_list.append(curr_allele)
    index += 1
  print("Data points: "+str(len(curr_sequence_list)))

f = open(processed_file_path+'test_all.tsv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f, delimiter='\t')
title = np.arange(3).tolist()
title[0] = "peptide"
title[1] = "allele"
title[2] = "label_num"
wr.writerow(title)

for index in range(len(curr_label_list)):
  data = []
  data.append(curr_sequence_list[index])
  data.append(curr_allele_list[index])
  data.append(curr_label_list[index])
  wr.writerow(data)

f.close()

Processing HLA-A02:01
Data points: 12141
Processing HLA-A03:01
Data points: 19485
Processing HLA-A11:01
Data points: 25873
Processing HLA-A02:03
Data points: 32169
Processing HLA-A31:01
Data points: 37872
Processing HLA-A02:06
Data points: 43494
Processing HLA-A68:02
Data points: 49034
